In [9]:
import os
import shutil
import subprocess
from subprocess import DEVNULL, STDOUT, PIPE
import pandas as pd
import functools
from rdkit import Chem
from rdkit.Chem import PandasTools
import oddt
from oddt.scoring.functions import NNScore
from oddt.scoring.functions import RFScore
from oddt.scoring.functions.PLECscore import PLECscore
from tqdm import tqdm
import multiprocessing
import time
from scripts.utilities import *
from tqdm import tqdm
from IPython.display import display

#NNScore.nnscore.train(home_dir = '/home/tony/CADD22/software/pdbbind', sf_pickle = '/home/tony/CADD22/software/NNScore_pdbbind2016.pickle', pdbbind_version=2016)
rescorers = {'nnscore':NNScore.nnscore()}
scorer = rescorers['nnscore']
scorer.gen_training_data(pdbbind_dir= '/home/tony/CADD22/software/pdbbind', pdbbind_versions=(2015, 2016))
#scorer.train(home_dir = '/home/tony/CADD22/software/pdbbind', sf_pickle = '/home/tony/CADD22/software/NNScore_pdbbind2016.pickle', pdbbind_version=2016)


Exception: There is no PDBbind set availabe

In [11]:
oddt_prot=next(oddt.toolkit.readfile('pdb', '/home/tony/CADD22/wocondock_performance_aldr/receptor_protoss_prepared.pdb'))
oddt_prot.protein = True
scorer.set_protein(oddt_prot)
re_scores = []
df = PandasTools.LoadSDF('/home/tony/CADD22/wocondock_performance_aldr/temp/clustering/bestpose_clustered.sdf', idName='Pose ID', molColName='Molecule', removeHs=False)

In [14]:
scorer.load()
for mol in tqdm(df['Molecule']):
    oddt_mol = oddt.toolkit.Molecule(mol)
    scored_mol = scorer.predict_ligand(oddt_mol)


No pickle, training new scoring function.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  92 out of 1000 | elapsed:  1.6min remaining: 15.4min
[Parallel(n_jobs=-1)]: Done 193 out of 1000 | elapsed:  3.0min remaining: 12.6min
[Parallel(n_jobs=-1)]: Done 294 out of 1000 | elapsed:  4.5min remaining: 10.7min
[Parallel(n_jobs=-1)]: Done 395 out of 1000 | elapsed:  5.9min remaining:  9.0min
[Parallel(n_jobs=-1)]: Done 496 out of 1000 | elapsed:  7.4min remaining:  7.5min
[Parallel(n_jobs=-1)]: Done 597 out of 1000 | elapsed:  8.9min remaining:  6.0min
[Parallel(n_jobs=-1)]: Done 698 out of 1000 | elapsed: 10.5min remaining:  4.6min
[Parallel(n_jobs=-1)]: Done 799 out of 1000 | elapsed: 12.1min remaining:  3.0min
[Parallel(n_jobs=-1)]: Done 900 out of 1000 | elapsed: 13.6min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 15.1min finished
Test set:	R2_score: 0.6377	Rp: 0.8161	RMSE: 1.3082	SD: 1.2583
Train set

AttributeError: 'NoneType' object has no attribute 'predict'

In [ ]:
def oddt_nnscore_rescoring(sdf, mp):
    tic = time.perf_counter()
    print('Rescoring with NNScore')
    rescorers = {'nnscore':NNScore.nnscore()}
    nnscore_rescoring_folder = rescoring_folder+'/nnscore_rescoring/'
    create_temp_folder(nnscore_rescoring_folder)
    scorer = rescorers['nnscore']
    pickle = software+'/NNScore_pdbbind2016.pickle'
    scorer = scorer.load(pickle)
    oddt_prot=next(oddt.toolkit.readfile('pdb', protein_file))
    oddt_prot.protein = True
    scorer.set_protein(oddt_prot)
    re_scores = []
    df = PandasTools.LoadSDF(sdf, idName='Pose ID', molColName='Molecule', removeHs=False)
    if mp == 0:
        for mol in tqdm(df['Molecule']):
            Chem.MolToMolFile(mol, nnscore_rescoring_folder+'/temp.sdf')
            oddt_lig = next(oddt.toolkit.readfile('sdf', nnscore_rescoring_folder+'/temp.sdf'))
            scored_mol = scorer.predict_ligand(oddt_lig)
            re_scores.append(float(scored_mol.data['nnscore']))
    return re_scores